In [5]:
import cv2
import numpy as np

In [6]:
cap = cv2.VideoCapture("volleyball_video_full_2.mp4")
# cap = cv2.VideoCapture("output/high_res_video/volleyball_5_mins.mp4")
# cap = cv2.VideoCapture("output/high_res_video/volleyball_clip.mp4")

In [7]:
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
subtractor = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=500, detectShadows=True)
# subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=True)
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
font_color = (0, 255, 255)
thickness = 1
line_type = cv2.LINE_AA
label_position = (10, 20)

In [8]:
scale_factor = 0.45
new_width = int(width * scale_factor)
new_height = int(height * scale_factor)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_background_subtraction.avi', fourcc, fps, (new_width*2, new_height*2))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (new_width, new_height))

    movement_mask = subtractor.apply(frame)
    movement_mask[movement_mask == 127] = 0
    movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2)))
    movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_RECT, (8, 12)))

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([10, 0, 0])
    upper_yellow = np.array([50, 255, 255])
    color_mask = cv2.inRange(frame_hsv, lower_yellow, upper_yellow)
    color_mask = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    color_mask = cv2.morphologyEx(color_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))

    contours, _ = cv2.findContours(color_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ball_mask = np.zeros_like(frame[:, :, 0])
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        center = (int(x), int(y))
        cv2.circle(ball_mask, center, int(radius) + 1, 255, -1)

    contours_movement, _ = cv2.findContours(movement_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_movement = sorted(contours_movement, key=cv2.contourArea, reverse=True)[:6]
    movement_mask = np.zeros_like(movement_mask)
    cv2.drawContours(movement_mask, contours_movement, -1, 255, thickness=cv2.FILLED)

    players_only = cv2.bitwise_and(frame, frame, mask=movement_mask)
    ball_only = cv2.bitwise_and(frame, frame, mask=ball_mask)
    ball_and_players = cv2.bitwise_or(players_only, ball_only)

    frame_display = frame.copy()
    players_display = players_only.copy()
    ball_display = ball_only.copy()
    combined_display = ball_and_players.copy()

    cv2.putText(frame_display, "Original", label_position, font, font_scale, font_color, thickness, line_type)
    cv2.putText(players_display, "Players Only", label_position, font, font_scale, font_color, thickness, line_type)
    cv2.putText(ball_display, "Ball Only", label_position, font, font_scale, font_color, thickness, line_type)
    cv2.putText(combined_display, "Ball + Players", label_position, font, font_scale, font_color, thickness, line_type)

    top_row = cv2.hconcat([frame_display, players_display])
    bottom_row = cv2.hconcat([ball_display, combined_display])
    display_grid = cv2.vconcat([top_row, bottom_row])

    cv2.imshow("Tracking View", display_grid)

    out.write(display_grid)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()